In [4]:
import os
import json
import psycopg2

from dotenv import load_dotenv
from urllib.parse import quote_plus
from dune_client.client import DuneClient

load_dotenv()
dune = DuneClient.from_env()

# Define the additional option with URL encoding
encoded_option = quote_plus(f"endpoint={os.getenv('DATABASE_ENDPOINT')}")

# Connect using psycopg2 with connection string
connection_string = f"postgresql://{os.getenv('DATABASE_USER')}:{os.getenv('DATABASE_PASSWORD')}@{os.getenv('DATABASE_HOST')}:{os.getenv('DATABASE_PORT')}/{os.getenv('DATABASE_NAME')}?options={encoded_option}"
conn = psycopg2.connect(connection_string)
cur = conn.cursor()

# Sample database operation: fetching the current date and time from the server
cur.execute("SELECT NOW();")
current_time = cur.fetchone()
print(f"Current database server time: {current_time[0].strftime('%Y-%m-%d %H:%M')} UTC")

Current database server time: 2024-05-12 17:03 UTC


In [ ]:
# This does NOT execute the query, only fetches most recent cached result
query_result = dune.get_latest_result(3695328)

In [ ]:
res = query_result.get_rows()

In [ ]:
len(res)

In [42]:
res[-1]

{'block_height': 824205,
 'block_time': '2024-01-03 21:04:46.000 UTC',
 'cat_number': 0,
 'fee': 0.00040834,
 'id': '0x98316dcb21daaa221865208fe0323616ee6dd84e6020b78bc6908e914ac03892',
 'input': [[0.00634426,
   None,
   824183,
   '0x97b4ecfe7c015aa35f3a2e0299be09a0dc3d02c6dbcff31a23c80f806a19af9a',
   11,
   ['3PhEFGZSE4JhMa8JPoX5rf9gUZnsDwCcJt',
    'OP_HASH160 f15d0dc87fc1cd6d2395870a7c6a6788854c03ec OP_EQUAL',
    'addr(3PhEFGZSE4JhMa8JPoX5rf9gUZnsDwCcJt)#m2rmu9t3',
    '0xa914f15d0dc87fc1cd6d2395870a7c6a6788854c03ec87',
    'scripthash'],
   ['0014d6128ca4880dc012f901129ee3491c2d1c343b26',
    '0x160014d6128ca4880dc012f901129ee3491c2d1c343b26'],
   4294967293,
   ['0x3044022065051b1b6479dd36eb921894b3173a0bc3768d75bfadbd473839147580f75d8302201d0a1952da156f37f05e3b2898c52fdd18fd133a47b2d4624098510597626a7501',
    '0x029ad68868c3175c8e7d62e831d5dc3830837352f5f3be8bd3e1a997c78e4cbd70']]],
 'input_count': 1,
 'lock_time': 21,
 'output': [[0,
   ['bc1p85ra9kv6a48yvk4mq4hx08wxk6t32td

In [62]:
# Create table
cur.execute("DROP TABLE cats")
cur.execute("""CREATE TABLE IF NOT EXISTS cats (
  id UUID PRIMARY KEY DEFAULT gen_random_uuid(),
  cat_number INT,
  lock_time INT,
  block_height INT,
  minted_at TIMESTAMP WITH TIME ZONE,
  minted_by VARCHAR(256),
  feeRate FLOAT,
  tx_hash VARCHAR(256),
  tx_fee FLOAT,
  tx_virtual_size INT,
  tx_input_count INT,
  tx_input JSONB,
  tx_output_count INT,
  tx_output JSONB
);
""")
conn.commit()

In [38]:
# Need this for gen_random_uuid()
cur.execute("CREATE EXTENSION IF NOT EXISTS pgcrypto;")
conn.commit()

In [48]:
insert_query = """
INSERT INTO public.cats (
  cat_number,
  lock_time,
  block_height,
  minted_at,
  minted_by,
  feerate,
  tx_hash,
  tx_input,
  tx_input_count,
  tx_output,
  tx_output_count,
  tx_fee,
  tx_virtual_size
) VALUES(%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s);
"""

In [64]:
for row in res[1:]:
    # TODO: Handle cases with OP_RETURN as first output, use tx_output[1][1][0] as minter for them

    # Construct INSERT for the normal case
    cur.execute(insert_query, (
        row["cat_number"],
        row["lock_time"],
        row["block_height"],
        row["block_time"],
        row["output"][0][1][0],
        100000000 * row["fee"] / row["virtual_size"],
        row["id"][2:],
        json.dumps(row["input"]),
        row["input_count"],
        json.dumps(row["output"]),
        row["output_count"],
        row["fee"],
        row["virtual_size"],
    ))
conn.commit()

In [47]:
res[0]

{'block_height': 824205,
 'block_time': '2024-01-03 21:04:46.000 UTC',
 'cat_number': 0,
 'fee': 0.00040834,
 'id': '0x98316dcb21daaa221865208fe0323616ee6dd84e6020b78bc6908e914ac03892',
 'input': [[0.00634426,
   None,
   824183,
   '0x97b4ecfe7c015aa35f3a2e0299be09a0dc3d02c6dbcff31a23c80f806a19af9a',
   11,
   ['3PhEFGZSE4JhMa8JPoX5rf9gUZnsDwCcJt',
    'OP_HASH160 f15d0dc87fc1cd6d2395870a7c6a6788854c03ec OP_EQUAL',
    'addr(3PhEFGZSE4JhMa8JPoX5rf9gUZnsDwCcJt)#m2rmu9t3',
    '0xa914f15d0dc87fc1cd6d2395870a7c6a6788854c03ec87',
    'scripthash'],
   ['0014d6128ca4880dc012f901129ee3491c2d1c343b26',
    '0x160014d6128ca4880dc012f901129ee3491c2d1c343b26'],
   4294967293,
   ['0x3044022065051b1b6479dd36eb921894b3173a0bc3768d75bfadbd473839147580f75d8302201d0a1952da156f37f05e3b2898c52fdd18fd133a47b2d4624098510597626a7501',
    '0x029ad68868c3175c8e7d62e831d5dc3830837352f5f3be8bd3e1a997c78e4cbd70']]],
 'input_count': 1,
 'lock_time': 21,
 'output': [[0,
   ['bc1p85ra9kv6a48yvk4mq4hx08wxk6t32td

In [52]:
100000000 * res[0]["fee"] / res[0]["virtual_size"]

230.70056497175142

In [2]:
"0x98316dcb21daaa221865208fe0323616ee6dd84e6020b78bc6908e914ac03892"[2:]

'98316dcb21daaa221865208fe0323616ee6dd84e6020b78bc6908e914ac03892'